In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [42]:
scoring_df = pd.read_csv('scoring_dataset.csv')
defense_df = pd.read_csv('defense_dataset.csv')
defense_diff_df = pd.read_csv('defense_diff_dataset.csv')

scoring_columns_to_keep = [
    'PLAYER', 'GP', 'MIN', 'PTS', 'DRIVES',
    'C&S\nPTS', 'C&S\nFG%', 
    'Pull Up\nPTS', 'Pull Up\nFG%', 
    'eFG%', 'Shot Quality', 
    'Assisted 2s\n PTS', 'Unassisted 2s\n PTS',
    'Assisted 3s\n PTS', 'Unassisted 3s\n PTS'
]

defense_columns_to_keep = [
    'Player', 'STL', 'BLK'
]

defense_diff_columns_to_keep = [
    'PLAYER', 'DIFF%'
]

cleaned_scoring = scoring_df[scoring_columns_to_keep].copy()
cleaned_defense = defense_df[defense_columns_to_keep].copy()
cleaned_defense_diff = defense_diff_df[defense_diff_columns_to_keep].copy()

cleaned_scoring.columns = [
    'Player', 'GP', 'MIN', 'PPG', 'Drives',
    'C&S PTS', 'C&S FG%',
    'PU PTS', 'PU FG%',
    'eFG%', 'Shot Quality',
    'Assisted 2s', 'Unassisted 2s',
    'Assisted 3s', 'Unassisted 3s'
]

cleaned_defense.columns = [
    'Player', 'Steals', 'Blocks'
]

cleaned_defense_diff.columns = [
    'Player', 'DIFF%'
]

percent_cols = ['C&S FG%', 'eFG%', 'PU FG%']

for col in percent_cols:
    cleaned_scoring[col] = cleaned_scoring[col].astype(str).str.rstrip('%').astype(float) / 100

Unassisted_PTS = cleaned_scoring['Unassisted 2s'] + cleaned_scoring['Unassisted 3s']
cleaned_scoring['Unassisted PTS'] = Unassisted_PTS
cleaned_scoring['Unassisted%'] = cleaned_scoring['Unassisted PTS'] / cleaned_scoring['PPG']
cleaned_scoring['C&S%'] = cleaned_scoring['C&S PTS'] / cleaned_scoring['PPG']
cleaned_scoring['PU%'] = cleaned_scoring['PU PTS'] / cleaned_scoring['PPG']
cleaned_scoring['Gap'] = cleaned_scoring['eFG%'] - cleaned_scoring['Shot Quality']
cleaned_defense_diff['DIFF%'] = cleaned_defense_diff['DIFF%'] / 100

cleaned = pd.merge(cleaned_scoring, cleaned_defense, on='Player', how='inner')
cleaned = pd.merge(cleaned, cleaned_defense_diff, on='Player', how='inner')


final_columns_to_drop = [
    'GP', 'MIN', 'PPG',
    'C&S PTS', 'C&S FG%',
    'PU PTS', 'PU FG%',
    'Assisted 2s', 'Unassisted 2s',
    'Assisted 3s', 'Unassisted 3s',
    'Unassisted PTS', 'Blocks',
    'eFG%', 'Shot Quality'
]

cleaned.drop(columns=final_columns_to_drop, inplace=True)
cleaned.set_index('Player', inplace=True)
cleaned


,Drives,Unassisted%,C&S%,PU%,Gap,Steals,DIFF%
Player,,,,,,,
Amen Thompson,7.5,0.358156,0.078014,0.070922,0.015,1.4,-0.055
Andrew Nembhard,8.4,0.359000,0.210000,0.280000,-0.004,1.2,0.013
Anfernee Simons,9.3,0.504663,0.259067,0.357513,0.011,0.9,0.044
Anthony Edwards,13.2,0.457609,0.126812,0.409420,0.007,1.2,-0.006
Austin Reaves,10.6,0.324257,0.237624,0.262376,0.036,1.1,0.033
...,...,...,...,...,...,...,...
Tyler Herro,13.9,0.371967,0.230126,0.338912,0.043,0.9,0.031
Tyrese Haliburton,10.8,0.458602,0.139785,0.489247,0.062,1.4,0.023
Tyrese Maxey,14.2,0.500000,0.133080,0.330798,-0.029,1.8,-0.004
